In [1]:
!nvidia-smi

Thu Nov 21 08:05:40 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.104      Driver Version: 410.104      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    32W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from wandb.keras import WandbCallback
import tensorflow as tf
import numpy as np
import wandb
import time

In [3]:
wandb.init(project='mixed-precision-tf-keras', name='without-mp')

W&B Run: https://app.wandb.ai/sayakpaul/mixed-precision-tf-keras/runs/mthp3618

In [4]:
print(tf.__version__)
print(tf.test.is_gpu_available())

2.0.0
True


In [5]:
DATA_PATH = '/home/jupyter/Mixed-Precision-Training-in-tf.keras-2.0/Data'

X_train = np.load(DATA_PATH + '/X_train.npy', allow_pickle=True)
y_train = np.load(DATA_PATH + '/y_train.npy', allow_pickle=True)

In [6]:
print(X_train.shape, y_train.shape)
print(X_train.dtype)

(6252, 224, 224, 3) (6252,)
float64


In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

In [8]:
IMG_SIZE = (224, 224)

@tf.function
def format_train_example(image, label):
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    image = tf.image.resize(image, IMG_SIZE)
    return image, tf.one_hot(label, len(np.unique(y_train)))

In [9]:
BATCH_SIZE = 80
AUTOTUNE = tf.data.experimental.AUTOTUNE
PREFETCH_COUNT = 8

train_dataset = train_dataset.shuffle(1024)
train_dataset = train_dataset.repeat()
train_dataset = train_dataset.map(format_train_example,
                                  num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(PREFETCH_COUNT)

In [10]:
def create_model(img_size=(224,224), num_class=5, train_base=True):
    # Accept float16 image inputs
    input_layer = Input(shape=(img_size[0],img_size[1],3))
    base = ResNet50(input_tensor=input_layer,
                    include_top=False,
                    weights="imagenet")
    base.trainable = train_base
    x = base.output
    x = GlobalAveragePooling2D()(x)
    # softmax only accepts float32 - need to manually cast (likely a bug)
    preds = Dense(num_class, activation="softmax")(x)
    return Model(inputs=input_layer, outputs=preds)

In [11]:
model = create_model()
opt = Adam()
model.compile(loss="categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

In [12]:
# Kickstart model training
start = time.time()
model.fit(train_dataset, 
         steps_per_epoch=X_train.shape[0]//BATCH_SIZE,
         epochs=5,
         callbacks=[WandbCallback()])
print("It took {} seconds".format(time.time() - start))

Train for 78 steps
Epoch 1/5
78/78 [==============================] - 84s 1s/step - loss: 0.6833 - accuracy: 0.5853
Epoch 2/5
78/78 [==============================] - 60s 776ms/step - loss: 0.8061 - accuracy: 0.5258
Epoch 3/5
78/78 [==============================] - 60s 772ms/step - loss: 0.9620 - accuracy: 0.4413
Epoch 4/5
78/78 [==============================] - 60s 775ms/step - loss: 0.9894 - accuracy: 0.4436
Epoch 5/5
78/78 [==============================] - 60s 775ms/step - loss: 1.0601 - accuracy: 0.3894
It took 329.9550199508667 seconds
